In [1]:
from mlpatterns.util._style_init_ import align_output_tables
align_output_tables('left')

[(< index)](../../Guide.ipynb)

# Linear Regression - univariate

## Summary

In [2]:
%run ./_summary_.ipynb

| Abbreviation | Architecture                                     |    Type    | Use                   | Limitation                               |
|:------------:|:-------------------------------------------------|:----------:|:----------------------|:-----------------------------------------|
    |  linreguni   | Polynomial in one variable (aka 'input feature') | Supervised | Real value prediction | Underfits non-linear data<br>(high bias) |


## Characteristics

<u>_Objective_</u>
- Tune a polynomial that takes the input feature, and returns a predicted output value

<u>_Learning algorithm_</u>
- Gradient descent

<u>_Parameters_</u>
- polynomial co-efficients, $\theta$

<u>_Hyper-parameters_</u>
- learning rate, alpha: $\alpha$
- polynomial degree / higher-order terms: $x_{j}^{p}$

## Algorithms

In [3]:
# Packages needed for the examples
import numpy as np

rng = np.random.default_rng(seed=42)

In [4]:
# Data
linreg_x = np.array([[2, 3, 5]])
linreg_y = np.array([[11, 17, 29]])

### Forward function / Prediction

> $\normalsize
\begin{align}
h_{\theta}(x) &= \theta_0 x_0 + \theta_1 x_1 \\
  &= \theta \cdot x \\
\end{align}
$

> where,
>
> $
\begin{align}
\theta &\text{ is a row vector of coefficients (parameters) of the polynomial } h_{\theta}(x) \\
x &\text{ is a col vector with } x_0 \text{ set to } 1
\end{align}
$

> **Example linreguni-1:**
>
> Given,
>
> $
\begin{align}
\theta &= \begin{bmatrix}
2 & 5
\end{bmatrix}
,\;
x = \begin{bmatrix}
1 \\
2
\end{bmatrix} \\
\end{align}
$

> Then,
>
> $
\begin{align}
\theta \cdot x &= \begin{bmatrix} (\theta_{[0,0]} \times x_{[0,0]}) + (\theta_{[0,1]} \times x_{[1,0]}) \end{bmatrix} \\
  \;&= \begin{bmatrix} (2 \times 1) + (5 \times 2) \end{bmatrix} \\
  \;&= \begin{bmatrix} 2 + 10 \end{bmatrix} \\
  \;&= \begin{bmatrix} 12 \end{bmatrix}
\end{align}
$


In [5]:
def linreg_h_theta(theta, features):
    """
    :param theta: Row vector of polynomial coefficients (i.e. parameters)
    :param features: Col vector of features, with x_0 set to 1 (i.e. bias term)
    :return: Estimate of y (i.e. y_hat)
    """

    y_hat = np.dot(theta, features)
    return y_hat

In [ ]:
# linreguni-fw Example

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, 0:1])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))
print('theta (shape): ', theta.shape)
print(theta, '\n')
print('x (shape): ', x.shape)
print(x, '\n')
print('X (shape) with bias terms: ', X.shape)
print(X, '\n')

y_hat = linreg_h_theta(theta, X)
print('y_hat (shape): ', y_hat.shape)
print(y_hat)

#### Vectorized (across samples)

> $\normalsize
\begin{align}
h_{\theta}(X) &= \theta \cdot X \\
\end{align}
$

> where,
>
> $
\begin{align}
\theta &\text{ is a row vector of coefficients (parameters) of the polynomial } h_{\theta} \\
X &\text{ is a } n \times m \text{ matrix of samples} \\
n &\text{ is the number of features; for univariate, } n = 2, (x_0,\;x_1) \\
m &\text{ is the number of samples}
\end{align}
$

> **Example linreguni-2:**
>
> Given,
>
> $
\begin{align}
\theta = \begin{bmatrix}
2 & 5
\end{bmatrix}
,\;
X = \begin{bmatrix}
1 & 1 & 1 \\
2 & 3 & 5 \\
\end{bmatrix}
\end{align}
$
>
> so $X$ contains three samples (the cols of $X$)


> Then,
>
> $
\begin{align}
\theta \cdot X &= \begin{bmatrix}
  (\theta_{[0,0]} \times X_{[0,0]}) + (\theta_{[0,1]} \times X_{[1,0]}) &
  (\theta_{[0,0]} \times X_{[0,1]}) + (\theta_{[0,1]} \times X_{[1,1]}) &
  (\theta_{[0,0]} \times X_{[0,2]}) + (\theta_{[0,1]} \times X_{[1,2]})
\end{bmatrix} \\
  \;&= \begin{bmatrix}
    (2 \times 1) + (5 \times 2) &
    (2 \times 1) + (5 \times 3) &
    (2 \times 1) + (5 \times 5)
  \end{bmatrix} \\
  \;&= \begin{bmatrix}
    (2 + 10) & (2 + 15) & (2 + 25)
  \end{bmatrix} \\
  \;&= \begin{bmatrix}
    12 & 17 & 27
  \end{bmatrix}
\end{align}
$
>
> so three predictions are returned (the cols of the result)


In [ ]:
# lingreguni-fw_VECT example

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, :])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))
print('theta (shape): ', theta.shape)
print(theta, '\n')
print('x (shape): ', x.shape)
print(x, '\n')
print('X (shape) with bias terms: ', X.shape)
print(X, '\n')

y_hat = linreg_h_theta(theta, X)
print('y_hat (shape): ', y_hat.shape)
print(y_hat)

### Loss function

> $\normalsize
\begin{align}
L_{\theta}(\hat{y}, y) &= \hat{y} - y
\end{align}
$


> where,
>
> $
\begin{align}
\hat{y} &\text{ is the prediction from } h_{\theta}(x) \\
y &\text{ is the training value that } h_{\theta}(x) \text{ should match}
\end{align}
$

In [6]:
def linreg_loss_theta(y_hat, y):
    """
    :param y_hat: the prediction from h_theta
    :param y: the training label to match
    :return: loss, or error, of the prediction
    """

    loss = y_hat - y
    return loss

In [ ]:
# linreguni-loss

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, 0:1])
y = np.array(linreg_y[:, 0:1])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))

y_hat = linreg_h_theta(theta, X)
loss = linreg_loss_theta(y_hat, y)

print('loss (shape): ', loss.shape)
print(loss, '\n')

In [ ]:
# linreguni-loss_VECT (across samples)

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, :])
y = np.array(linreg_y[:, :])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))

y_hat = linreg_h_theta(theta, X)
loss = linreg_loss_theta(y_hat, y)

print('loss (shape): ', loss.shape)
print(loss, '\n')

### Cost function

> $\normalsize
\begin{align}
    \;& \;&
    \;&\small \underline{Normal}&
    \;& \;&
    \;&\small \underline{Vectorized} \\ \\
J(\theta) &=&
    \;&\frac{1}{2m}\sum\limits_{i=1}^{m} \left[h_{\theta}(x^{(i)}) - y^{(i)}\right]^2 &
    \;&\Rightarrow&
    \;&\frac{1}{2m} \left[ \left( \theta X - Y \right) \cdot \left( \theta X - Y \right)^{T} \right] \\
    \;&=&
    \;&\frac{1}{2m}\sum\limits_{i=1}^{m} L_{\theta}(\hat{y}^{(i)}, y^{(i)})^2 &
    \;&\Rightarrow&
    \;&\frac{1}{2m} \left( L_{\theta} \cdot L_{\theta}^{T} \right) \\
\end{align}
$
>
> where,
>
> $
\begin{align}
m &\text{ is the number of samples} \\
\\ \text{and} \\
\; &\text{vectorization is across samples, } m
\end{align}
$

In [7]:
def linreg_cost_from_loss(L_theta):
    """
    :param L_theta: matrix (1 x m) of errors (loss)
    :return: cost (mean squared error) to be minimized
    """

    m = L_theta.shape[1]
    cost = np.dot(L_theta, L_theta.T) / (2 * m)

    return cost

In [8]:
def linreg_cost_from_inputs(theta, X, Y):
    """
    :param theta: row vector (1 x n) of polynomial coefficients (i.e. parameters)
    :param X: n x m matrix of features, with x_0's set to 1 (i.e. bias terms)
    :param Y: row vector (1 x m) of target values
    :return: cost (mean squared error) to be minimized
    """

    m = X.shape[1]
    cost = np.dot(np.dot(theta, X) - y, (np.dot(theta, X) - y).T) / (2 * m)

    return cost

In [ ]:
# linreguni-cost

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, :])
y = np.array(linreg_y[:, :])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))

y_hat = linreg_h_theta(theta, X)
L_theta = linreg_loss_theta(y_hat, y)

cost_from_loss = linreg_cost_from_loss(L_theta)
print('cost_from_loss (shape): ', cost_from_loss.shape)
print(cost_from_loss, '\n')

cost_from_inputs = linreg_cost_from_inputs(theta, X, y)
print('cost_from_inputs (shape): ', cost_from_inputs.shape)
print(cost_from_inputs, '\n')


### Backward function / Learning / Parameter update

> $\normalsize
\begin{align}
    \;& \;&
        \;&\small \underline{Normal}&
        \;& \;&
        \;&\small \underline{Vectorized} \\ \\
dJ(\theta)
        &=&
        \;&\frac{1}{m}\sum\limits_{i=1}^{m} L_{\theta}(\hat{y}^{(i)}, y^{(i)})x_{j}^{(i)}&
        \;&\Rightarrow&
        \;&\frac{1}{m} \left( L_{\theta} \cdot X^{T}\right) \\
    \;&=&
        \;&\frac{1}{m}\sum\limits_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})x_{j}^{(i)}&
        \;& \;&
        \;& \; \\
    \;&=&
        \;&\frac{1}{m}\sum\limits_{i=1}^{m} \left[h_{\theta}(x^{(i)}) - y^{(i)}\right]x_{j}^{(i)}&
        \;&\Rightarrow&
        \;&\frac{1}{m} \; \left[ (\theta X - Y) \cdot X^{T} \right]
\end{align}
$

> $\normalsize
\begin{align}
    \;& \;&
        \;&\small \underline{Normal}&
        \;& \;&
        \;&\small \underline{Vectorized} \\ \\
\theta_{j}
        &:= \theta_{j} \; -&
        \;&\alpha \; dJ(\theta)&
        \;& \;&
        \;& \; \\
    \;&:= \theta_{j} \; -&
        \;&\boxed{\alpha \left( \frac{1}{m}\sum\limits_{i=1}^{m} \left[h_{\theta}(x^{(i)}) - y^{(i)}\right]x_{j}^{(i)} \right) }&
        \;&\Rightarrow&
        \;&\boxed{\alpha \left( \frac{1}{m} \; \left[ (\theta X - Y) \cdot X^{T} \right] \right) } \\
    \;& \;&
        \; &\small{\text{OR}}&
        \;& \;&
        \;& \; \\
    \;& \;&
        \;&\boxed{\alpha \left( \frac{1}{m}\sum\limits_{i=1}^{m} L_{\theta}(\hat{y}^{(i)}, y^{(i)})x_{j}^{(i)} \right) }&
        \;&\Rightarrow&
        \;&\boxed{\alpha \left( \frac{1}{m} \left[ L_{\theta} \cdot X^{T}\right] \right)}
\end{align}
$

> where,
>
> $
\begin{align}
\theta &\text{ is a row vector of } n \text{ parameters, i.e. polynomial coefficients} \\
X &\text{ is a } n \times m \text{ matrix of samples} \\
Y &\text{ is a row vector of } m \text{ target values} \\
m &\text{ is the number of samples} \\
n &\text{ is the number of features (incl. bias terms), i.e. polynomial terms} \\
i &\text{ is the index } (1..m) \text{ of the samples} \\
j &\text{ is the index } (1..n) \text{ of the features, i.e. polynomial terms} \\
\alpha &\text{ is the learning rate}
\end{align}
$

In [9]:
def linreg_grad_from_loss(L_theta, X):
    """
    :param L_theta: row vector (1 x m) of errors (loss)
    :param X: n x m matrix of input features
    :return: gradient at the current theta
    """

    m = L_theta.shape[1]
    grad = np.dot(L_theta, X.T) / m

    return grad

In [10]:
def linreg_grad_from_inputs(theta, X, Y):
    """
    :param theta: row vector (1 x n) of parameters
    :param X: n x m matrix of input features
    :param Y: row vector (1 x m) of target values
    :return: gradient at the current theta
    """

    m = X.shape[1]
    grad = np.dot(np.dot(theta, X) - Y, X.T) / m

    return grad

In [11]:
def linreg_grad_validate(theta, X, Y, epsilon=1e-4):
    """
    :param theta: row vector of parameters
    :param X: n x m matrix of inputs
    :param Y: row vector of target values
    :param epsilon: real value << 0
    :return: row vector gradient estimates for cost function at current theta
    """

    n = theta.shape[1]
    grad_validation = np.zeros(theta.shape)

    for i in range(n):
        theta_plus_epsilon = np.copy(theta)
        theta_minus_epsilon = np.copy(theta)
        theta_plus_epsilon[0, i] = theta[0, i] + epsilon
        theta_minus_epsilon[0, i] = theta[0, i] - epsilon

        cost_plus_epsilon = linreg_cost_from_inputs(theta_plus_epsilon, X, Y)
        cost_minus_epsilon = linreg_cost_from_inputs(theta_minus_epsilon, X, Y)

        grad_validation[0, i] = (cost_plus_epsilon - cost_minus_epsilon) / (2 * epsilon)

    return grad_validation

In [ ]:
# linreguni-grad

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, :])
y = np.array(linreg_y[:, :])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))

y_hat = linreg_h_theta(theta, X)
L_theta = linreg_loss_theta(y_hat, y)

grad_from_loss = linreg_grad_from_loss(L_theta, X)
print('grad_from_loss (shape): ', grad_from_loss.shape)
print(grad_from_loss, '\n')

grad_from_inputs = linreg_grad_from_inputs(theta, X, y)
print('grad_from_inputs (shape): ', grad_from_inputs.shape)
print(grad_from_inputs, '\n')

grad_validation = linreg_grad_validate(theta, X, y)
print('grad_validation (shape): ', grad_validation.shape)
print(grad_validation, '\n')


In [12]:
def linreg_update_params(theta, grad, alpha):
    """
    :param theta: row vector (1 x n) of current parameters
    :param grad: row vector (1 x n) of gradients
    :param alpha: learning rate as real number
    :return: row vector of updated parmeters
    """

    params = theta - alpha * grad

    return params

In [ ]:
# linreguni-param-update

theta = np.array([[2., 5.]])
x = np.array(linreg_x[:, :])
y = np.array(linreg_y[:, :])
x_bias = np.ones((1, x.shape[1]))
X = np.vstack((x_bias, x))
alpha = 0.01

grad_from_inputs = linreg_grad_from_inputs(theta, X, y)
print('grad_from_inputs (shape): ', grad_from_inputs.shape)
print(grad_from_inputs, '\n')

params = linreg_update_params(theta, grad_from_inputs, alpha)
print('theta (shape): ', theta.shape)
print(theta, '\n')
print('params (shape): ', params.shape)
print(params, '\n')
